# Importing necessary libraries

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, AveragePooling2D, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D, Input, Concatenate, Lambda, ZeroPadding2D, Add
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import backend as K
import typing
from keras.models import load_model
from tensorflow.keras.callbacks import TensorBoard
import time
from sklearn.model_selection import train_test_split
from pathlib import Path
import os
import random

# Setting TensorFlow environment

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.config.run_functions_eagerly(True)

# Initializing WordNetLemmatizer

In [ ]:
lemmatizer = WordNetLemmatizer()

# Getting the absolute path of the current file's parent directory

In [ ]:
path = Path(__file__).parent.absolute()
str_path = str(path)

# Function for preprocessing

In [ ]:
def preprocess(dataset = 'dataset_22.json'):
    intents = json.loads(open(dataset, encoding='utf-8').read())
    words = []
    classes = []
    documents = []
    ignore_letters = ['?', '!', '.', ","]

    for intent in intents['intents']:
        for pattern in intent['patterns']:
            word_list = nltk.word_tokenize(pattern)
            words.extend(word_list)
            documents.append((word_list, intent['tag']))
            if intent['tag'] not in classes:
                classes.append(intent['tag'])

    #Lexicon of words and classes
    words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]

    words = sorted(set(words))
    classes = sorted(set(classes))

    stopwords = nltk.corpus.stopwords.words('english')
    ignore_letters = ['?', '!', '.', ","]

    bag = []
    for document in documents:
        word_patterns = document[0]
        # word_patterns -> sentence with lower words + lemmatize
        word_patterns = [lemmatizer.lemmatize((word.lower())) for word in word_patterns]
        #word_patterns_without_stopwords = [lemmatizer.lemmatize(word) for word in word_patterns if word not in stopwords]
        #word_patt = [lemmatizer.lemmatize(word) for word in word_patterns_without_stopwords if word not in ignore_letters]
        word_patterns_ignore_letters = [lemmatizer.lemmatize(word) for word in word_patterns if word not in ignore_letters]
        bag.append(word_patterns_ignore_letters)

    class_pattern = []
    for document in documents:
        classes = document[1]
        class_pattern.append(classes)

    return bag, class_pattern

# Function for creating TaggedDocument

In [ ]:
def tagged_document(list_of_list_of_words):
    for i, list_of_words in enumerate(list_of_list_of_words):
        yield TaggedDocument(list_of_words, [i])

# Function to build the model

In [ ]:
def build_model(data,vc,ep):
    data_for_training = list(tagged_document(data))
    model = Doc2Vec(vector_size=vc,min_count=2, epochs=ep)
    model.build_vocab(data_for_training)
    model.train(data_for_training,total_examples=model.corpus_count,epochs=model.epochs)

    return model

# Function for training vectors

In [ ]:
def tain_vec(bag, voc_size1, epochs):
    model_x = build_model(bag,voc_size1,epochs)
    train_x_dv = []
    for i in range(len(model_x.dv)):
        train_x_dv.append(model_x.dv[i])

    return train_x_dv, model_x

# Function for training the model

In [ ]:
def train_model(bag, class_pattern, size1, size2, width, heigth, channel, num_ep):
    train_x_dv, model_x = tain_vec(bag[size1:size2], width*heigth*channel, num_ep)

    num_of_classes = []
    for i in range(size1,size2):
        if class_pattern[i] not in num_of_classes:
            num_of_classes.append(class_pattern[i])

    train_y = []
    for i in range(len(num_of_classes)):
        train_y.append(i)

    y_train = []
    for i in range(size1,size2):
        for j in range(len(num_of_classes)):
            if class_pattern[i] == num_of_classes[j]:
                y_train.append(train_y[j])

    x_i = []
    for i in range(size1,size2):
        x = np.reshape(train_x_dv[i], (width, heigth, channel))
        x_i.append(x)
    x_2 = np.array(x_i)

    return x_2, y_train, model_x, num_of_classes

# LeNet5 model

In [ ]:
def LeNet_5(max_lebels = 11):
    model = Sequential()

    # C1: (None,32,32,1) -> (None,28,28,6).
    model.add(
        Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=(32, 32, 1), padding='valid'))

    # P1: (None,28,28,6) -> (None,14,14,6).
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

    # C2: (None,14,14,6) -> (None,10,10,16).
    model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))

    # P2: (None,10,10,16) -> (None,5,5,16).
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

    # Flatten: (None,5,5,16) -> (None, 400).
    model.add(Flatten())

    # FC1: (None, 400) -> (None,120).
    model.add(Dense(120, activation='tanh'))

    # FC2: (None,120) -> (None,84).
    model.add(Dense(84, activation='tanh'))

    # FC3: (None,84) -> (None,10).
    model.add(Dense(max_lebels, activation='softmax'))

    # Compile the model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

    return model

# Grouped convolution function for AlexNet

In [ ]:
def grouped_conv(input_val, name, half, filters, kernel_size, strides=1, padding='valid'):
    """
        Performs a grouped convolution.

        Parameters:
        -input_val: previous layer.
        -name: name of the convolution.
        -half: Number of channels for each convolution.
        -filters: Number of filters for each convolution.
        -kernel_size: Kernel size used for each convolution.
        -strides: stride. Default value is 1.
        -padding: 'valid'(default) or 'same'.

        Returns:
        -conv: concatenation of the 2 previous convolution layer.

    """
    input_val_1 = Lambda(lambda x: x[:, :, :, :half])(input_val)
    input_val_2 = Lambda(lambda x: x[:, :, :, half:])(input_val)

    conv_1 = Conv2D(filters=filters,
                    kernel_size=kernel_size,
                    padding=padding,
                    activation='relu',
                    name=name + '_1')(input_val_1)

    conv_2 = Conv2D(filters=filters,
                    kernel_size=kernel_size,
                    padding=padding,
                    activation='relu',
                    name=name + '_2')(input_val_2)

    conv = Concatenate(name=name)([conv_1, conv_2])

    return conv

# AlexNet model

In [ ]:
def AlexNet(max_lebels = 11):
    x = Input((227, 227, 3))

    conv1 = Conv2D(filters=96,
                   kernel_size=(11, 11),
                   strides=4,
                   activation='relu',
                   name='conv1')(x)

    pool1 = MaxPooling2D(pool_size=3,
                         strides=2)(conv1)

    conv2 = grouped_conv(input_val=pool1,
                         name='conv2',
                         half=48,
                         filters=128,
                         kernel_size=5,
                         padding='same')

    pool2 = MaxPooling2D(pool_size=3,
                         strides=2)(conv2)

    conv3 = Conv2D(filters=384,
                   kernel_size=(3, 3),
                   padding='same',
                   activation='relu',
                   name='conv3')(pool2)

    conv4 = grouped_conv(input_val=conv3,
                         name='conv4',
                         half=192,
                         filters=192,
                         kernel_size=3,
                         padding='same')

    conv5 = grouped_conv(input_val=conv4,
                         name='conv5',
                         half=192,
                         filters=128,
                         kernel_size=3,
                         padding='same')

    pool5 = MaxPooling2D(pool_size=3,
                         strides=2)(conv5)

    flatten = Flatten()(pool5)

    fc6 = Dense(4096, activation='relu', name='fc6')(flatten)

    fc7 = Dense(4096, activation='relu', name='fc7')(fc6)

    fc8 = Dense(max_lebels, activation='softmax', name='fc8')(fc7)

    model = Model(inputs=x, outputs=fc8)

    return model

# Block function for ResNet

In [ ]:
def block(X: tf.Tensor, kernel_size: int, filters: typing.List[int], stage_no: int, block_name: str, is_conv_layer: bool = False, stride: int = 2) -> tf.Tensor:
    """
    Block for residual network.
    Arguments:
    X             -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    kernel_size   -- integer, specifying the shape of the middle CONV's window for the main path
    filters       -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage_no      -- integer, used to name the layers, depending on their position in the network
    block_name    -- string/character, used to name the layers, depending on their position in the network
    is_conv_layer -- to identiy if identity downsample is needed
    stride        -- integer specifying the stride to be used

    Returns:
    X             -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """

    # names
    conv_name_base = "res" + str(stage_no) + block_name + "_branch"
    bn_name_base = "bn" + str(stage_no) + block_name + "_branch"

    # filters
    F1, F2, F3 = filters

    # save the input value for shortcut.
    X_shortcut = X

    #  First component
    # NOTE: if conv_layer, you need to do downsampling
    X = Conv2D(
        filters=F1,
        kernel_size=(1, 1),
        strides=(stride, stride) if is_conv_layer else (1, 1),
        padding="valid",
        name=conv_name_base + "2a",
        kernel_initializer="glorot_uniform",
    )(X)
    X = BatchNormalization(axis=3, name=bn_name_base + "2a")(X)
    X = Activation("relu")(X)

    # Second component
    X = Conv2D(
        filters=F2,
        kernel_size=(kernel_size, kernel_size),
        strides=(1, 1),
        padding="same",
        name=conv_name_base + "2b",
        kernel_initializer="glorot_uniform",
    )(X)
    X = BatchNormalization(axis=3, name=bn_name_base + "2b")(X)
    X = Activation("relu")(X)

    # Third component
    X = Conv2D(
        filters=F3,
        kernel_size=(1, 1),
        strides=(1, 1),
        padding="valid",
        name=conv_name_base + "2c",
        kernel_initializer="glorot_uniform",
    )(X)
    X = BatchNormalization(axis=3, name=bn_name_base + "2c")(X)

    # NOTE: if is_conv_layer, you need to do downsampling the X_shortcut to match the output (X) channel
    #       so it can be added together
    if is_conv_layer:
        X_shortcut = Conv2D(
            filters=F3,
            kernel_size=(1, 1),
            strides=(stride, stride),
            padding="valid",
            name=conv_name_base + "1",
            kernel_initializer="glorot_uniform",
        )(X_shortcut)
        X_shortcut = BatchNormalization(axis=3, name=bn_name_base + "1")(X_shortcut)

    # Shortcut value
    X = Add()([X, X_shortcut])
    X = Activation("relu")(X)

    return X

# ResNet model

In [ ]:
def ResNet(name: str, layers: typing.List[int], input_shape: typing.Tuple[int] = (64, 64, 3), classes: int = 6) -> Model:
    """
    Implementation of the popular ResNet architecture.
    Arguments:
    name        -- name of the architecture
    layers      -- number of blocks per layer
    input_shape -- shape of the images of the dataset
    classes     -- integer, number of classes
    Returns:
    model       -- a Model() instance in Keras
    Model Architecture:
    Resnet50:
        CONV2D -> BATCHNORM -> RELU -> MAXPOOL  // conv1
            -> CONVBLOCK -> IDBLOCK * 2         // conv2_x
            -> CONVBLOCK -> IDBLOCK * 3         // conv3_x
            -> CONVBLOCK -> IDBLOCK * 5         // conv4_x
            -> CONVBLOCK -> IDBLOCK * 2         // conv5_x
            -> AVGPOOL
            -> TOPLAYER
    Resnet101:
        CONV2D -> BATCHNORM -> RELU -> MAXPOOL  // conv1
            -> CONVBLOCK -> IDBLOCK * 2         // conv2_x
            -> CONVBLOCK -> IDBLOCK * 3         // conv3_x
            -> CONVBLOCK -> IDBLOCK * 22        // conv4_x
            -> CONVBLOCK -> IDBLOCK * 2         // conv5_x
            -> AVGPOOL
            -> TOPLAYER
    Resnet152:
        CONV2D -> BATCHNORM -> RELU -> MAXPOOL  // conv1
            -> CONVBLOCK -> IDBLOCK * 2         // conv2_x
            -> CONVBLOCK -> IDBLOCK * 7         // conv3_x
            -> CONVBLOCK -> IDBLOCK * 35        // conv4_x
            -> CONVBLOCK -> IDBLOCK * 2         // conv5_x
            -> AVGPOOL
            -> TOPLAYER
    """

    # get layers (layer1 is always the same so no need to provide)
    layer2, layer3, layer4, layer5 = layers

    # convert input shape into tensor
    X_input = Input(input_shape)

    # zero-padding
    X = ZeroPadding2D((3, 3))(X_input)

    # conv1
    X = Conv2D(
        filters = 64,
        kernel_size = (7, 7),
        strides = (2, 2),
        name = "conv1",
        kernel_initializer = "glorot_uniform",
    )(X)
    X = BatchNormalization(axis = 3, name = "bn_conv1")(X)
    X = Activation("relu")(X)
    X = MaxPooling2D((3, 3), strides = (2, 2))(X)

    # conv2_x
    X = make_layer(X, layers = layer2, kernel_size = 3, filters = [64, 64, 256], stride = 1, stage_no = 2)

    # conv3_x
    X = make_layer(X, layers = layer3, kernel_size = 3, filters = [128, 128, 512], stride = 2, stage_no = 3)

    # conv4_x
    X = make_layer(X, layers = layer4, kernel_size = 3, filters = [256, 256, 1024], stride = 2, stage_no = 4)

    # conv5_x
    X = make_layer(X, layers = layer5, kernel_size = 3, filters = [512, 512, 2048], stride = 1, stage_no = 5)

    # average pooling
    X = AveragePooling2D((2, 2), name = "avg_pool")(X)

    # output layer
    X = Flatten()(X)
    X = Dense(
        classes,
        activation = "softmax",
        name="fc" + str(classes),
        kernel_initializer = "glorot_uniform"
    )(X)

    model = Model(inputs = X_input, outputs = X, name = name)
    return model

# Make layer function for ResNet

In [ ]:
def make_layer(X: tf.Tensor, layers: int, kernel_size: int, filters: typing.List[int], stride: int, stage_no: int) -> tf.Tensor:
    """
    Method to create one conv-identity layer for ResNet.
    Arguments:
    X           -- input tensor
    layers      -- number of blocks per layer
    kernel_size -- size of the kernel for the block
    filters     -- number of filters/channels
    stride      -- number of stride for downsampling the input
    stage_no    -- stage number just to name the layer
    Returns:
    X           -- output tensor
    """

    # create convolution block
    X = block(
        X,
        kernel_size = kernel_size,
        filters = filters,
        stage_no = stage_no,
        block_name = "a",
        is_conv_layer = True,
        stride = stride
    )

    # create identity block
    block_name_ordinal = ord("b")
    for _ in range(layers - 1):
        X = block(
            X,
            kernel_size = kernel_size,
            filters =  filters,
            stage_no = stage_no,
            block_name = chr(block_name_ordinal)
        )
        block_name_ordinal += 1

    return X

# VGGNet model

In [ ]:
def VGGNet(name: str, architecture: typing.List[ typing.Union[int, str] ], input_shape: typing.Tuple[int], classes: int = 1000) -> Model:
    """
    Implementation of the VGGNet architecture.
    Arguments:
    name         -- name of the architecture
    architecture -- number of output channel per convolution layers in VGGNet
    input_shape  -- shape of the images of the dataset
    classes      -- integer, number of classes
    Returns:
    model        -- a Model() instance in Keras
    """

    # convert input shape into tensor
    X_input = Input(input_shape)

    # make convolution layers
    X = make_conv_layer(X_input, architecture)

    # flatten the output and make fully connected layers
    X = Flatten()(X)
    X = make_dense_layer(X, 4096)
    X = make_dense_layer(X, 4096)

    # classification layer
    X = Dense(units = classes, activation = "softmax")(X)

    model = Model(inputs = X_input, outputs = X, name = name)
    return model

# Make convolutional layer function for VGGNet

In [ ]:
def make_conv_layer(X: tf.Tensor, architecture: typing.List[ typing.Union[int, str] ], activation: str = 'relu') -> tf.Tensor:
    """
    Method to create convolution layers for VGGNet.
    In VGGNet
        - Kernal is always 3x3 for conv-layer with padding 1 and stride 1.
        - 2x2 kernel for max pooling with stride of 2.
    Arguments:
    X            -- input tensor
    architecture -- number of output channel per convolution layers in VGGNet
    activation   -- type of activation method
    Returns:
    X           -- output tensor
    """

    for output in architecture:

        # convolution layer
        if type(output) == int:
            out_channels = output

            X = Conv2D(
                filters = out_channels,
                kernel_size = (3, 3),
                strides = (1, 1),
                padding = "same"
            )(X)
            X = BatchNormalization()(X)
            X = Activation(activation)(X)

            # relu activation is added (by default activation) so that all the
            # negative values are not passed to the next layer

        # max-pooling layer
        else:
            X = MaxPooling2D(
                pool_size = (2, 2),
                strides = (2, 2)
            )(X)

    return X

# Make dense layer function for VGGNet

In [ ]:
def make_dense_layer(X: tf.Tensor, output_units: int, dropout = 0.5, activation = 'relu') -> tf.Tensor:
    """
    Method to create dense layer for VGGNet.
    Arguments:
    X            -- input tensor
    output_units -- output tensor size
    dropout      -- dropout value for regularization
    activation   -- type of activation method
    Returns:
    X            -- input tensor
    """

    X = Dense(units = output_units)(X)
    X = BatchNormalization()(X)
    X = Activation(activation)(X)
    X = Dropout(dropout)(X)

    return X

# Custom F1 score function

In [ ]:
def custom_f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))

        recall = TP / (Positives + K.epsilon())
        return recall

    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

        precision = TP / (Pred_Positives + K.epsilon())
        return precision

    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)

    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

# Function for training models

In [ ]:
def training_models(size1 = 0, width = 28, heigth = 28, channel = 1, num_ep = 100, Epochs = 200, model_name = "LeNet5", dataset = 'intents.json'):
    start_time1 = time.time()
    bag, class_pattern = preprocess(dataset)

    size2 = len(bag)

    NAME = model_name+"-test-{}".format(int(Epochs))
    tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
    x_2, y_train, model_x, num_of_classes = train_model(bag, class_pattern, size1, size2, width, heigth, channel, num_ep)
    end_time1 = time.time()
    model_x.save('Doc2Vec_'+model_name+'-test.h5')
    pickle.dump(num_of_classes, open('num_of_classes_'+model_name+'-test.pkl', 'wb'))
    max_lebels = max(y_train)

    X_train, X_test, y_train2, y_test = train_test_split(x_2, y_train, test_size=0.33, random_state=42)

    y_train2 = np.array(y_train2)
    X_train = np.pad(X_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
    y_train2 = np.pad(y_train2, (0,0), 'constant')

    if model_name == "LeNet5":
        model = LeNet_5(max_lebels+1)
    elif model_name == "AlexNet":
        model = AlexNet(max_lebels + 1)
    elif model_name == "ResNet":
        model = ResNet(name="Resnet50", layers=[3, 4, 6, 3], input_shape=(224, 224, 3), classes=max_lebels + 1)
    elif model_name == "VGGNet":
        VGG_types = {
            'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
            'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
            'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
            'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512,
                      'M'],
        }
        model = VGGNet(name = "VGGNet16", architecture = VGG_types["VGG16"], input_shape=(224, 224, 3), classes = max_lebels+1)

    model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics = ['accuracy', custom_f1])
    #model.summary()

    hist = model.fit(X_train, y_train2, epochs=Epochs, validation_split=0.3, callbacks=[tensorboard])
    model.save('chatbotmodel_'+model_name+'-test.h5', hist)
    print('Done')

    y_test = np.array(y_test)
    X_test = np.pad(X_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')
    y_test = np.pad(y_test, (0,0), 'constant')

    str_path1 = os.path.join(str_path, "LeNet5_eval"+NAME+".txt")
    f2 = open(str_path1, "w", encoding = "utf-8")
    res3 = model.evaluate(X_test, y_test, batch_size=150)
    f2.write("--"+NAME+"\n")
    f2.write("Preprocess : "+str(end_time1-start_time1)+"\n")
    f2.write(str(res3)+"\n")
    f2.close()

# Function for chatbot

In [ ]:
def chatbot(model_name = "LeNet5", width = 28, heigth = 28, channel = 1, dataset = 'intents.json', mode = 'manual', questions = ["Hi", "How are you", "Goodbye"]):
    model_x = Doc2Vec.load('Doc2Vec_'+model_name+'-test.h5')
    num_of_classes = pickle.load(open('num_of_classes_'+model_name+'-test.pkl', 'rb'))
    model = load_model('chatbotmodel_'+model_name+'-test.h5', custom_objects={"custom_f1": custom_f1})

    intents = json.loads(open(dataset, encoding='utf-8').read())
    print("Chatbot in running. Please ask a question")

    if mode == "auto":
        for i in range(len(questions)):
            res = (model_x.infer_vector(questions[i].split()))
            print("Me : "+questions[i])  #Question

            x = np.reshape(res, (1,width, heigth, channel))
            x2 = np.pad(x, ((0,0),(2,2),(2,2),(0,0)), 'constant')

            res2 = model.predict(x2)
            #print(str(np.argmax(res2))+" - "+str(np.max(res2)))  # Number of class - Probability

            if np.max(res2) > 0.25:
                #print(num_of_classes[np.argmax(res2)])  #Class or Tag
                tag = num_of_classes[np.argmax(res2)]
                list_of_inttents = intents['intents']
                for i in list_of_inttents:
                    if i['tag'] == tag:
                        print("Chatbot : "+random.choice(i['responses']))  #Response
            else :
                print("I don't understand!")
    elif mode == "manual":
        stop = False
        while not stop:
            message = input("Me : ")
            if message == 'STOP':
                stop = True
            else:
                tokenized_message = nltk.word_tokenize(message)
                print(nltk.word_tokenize(message))

                res = (model_x.infer_vector(tokenized_message))
                #print("Me : " + questions[i])  # Question

                x = np.reshape(res, (1, width, heigth, channel))
                x2 = np.pad(x, ((0, 0), (2, 2), (2, 2), (0, 0)), 'constant')

                res2 = model.predict(x2)
                # print(str(np.argmax(res2))+" - "+str(np.max(res2)))  # Number of class - Probability

                if np.max(res2) > 0.25:
                    # print(num_of_classes[np.argmax(res2)])  #Class or Tag
                    tag = num_of_classes[np.argmax(res2)]
                    list_of_inttents = intents['intents']
                    for i in list_of_inttents:
                        if i['tag'] == tag:
                            print("Chatbot : " + random.choice(i['responses']))  # Response
                else:
                    print("I don't understand!")

# Timing the execution

In [ ]:
start_time1 = time.time()

# Preprocessing dataset

In [ ]:
bag, class_pattern = preprocess('intents.json')

# Training

## LeNet5

In [ ]:
training_models(size1=0, width=28, heigth=28, channel=1, num_ep=100, Epochs=200, model_name="LeNet5", dataset='intents.json')

## AlexNet

In [ ]:
training_models(size1=0, width=223, heigth=223, channel=3, num_ep=100, Epochs=200, model_name="AlexNet", dataset='intents.json')

## ResNet

In [ ]:
training_models(size1=0, width=220, heigth=220, channel=3, num_ep=100, Epochs=15, model_name="ResNet", dataset='intents.json')

## VGGNet

In [ ]:
training_models(size1=0, width=220, heigth=220, channel=3, num_ep=100, Epochs=15, model_name="VGGNet", dataset='intents.json')

In [ ]:
print(f"Total execution time: {time.time() - start_time1} seconds")

# Chatbot

## LeNet5

In [ ]:
chatbot(model_name="LeNet5", width=28, heigth=28, channel=1, dataset='intents.json', mode='manual', questions=["Hi", "How are you", "Goodbye"])

## AlexNet

In [ ]:
chatbot(model_name="AlexNet", width=223, heigth=223, channel=3, dataset='intents.json', mode='manual', questions=["Hi", "How are you", "Goodbye"])

## ResNet

In [ ]:
chatbot(model_name="ResNet", width=220, heigth=220, channel=3, dataset='intents.json', mode='manual', questions=["Hi", "How are you", "Goodbye"])

## VGGNet

In [ ]:
chatbot(model_name="VGGNet", width=220, heigth=220, channel=3, dataset='intents.json')